# Imports

In [6]:
import pm4py
import pandas as pd
from TreeToTptp import TreeToTptp
from ProverWrapper import use_provers
from create_thesis_with_2_spec import create_thesis_with_2_spec, LogicFunctions
import os

In [7]:
def print_provers(provers):
    print(provers[0])
    print(f'Vampire: {provers[1]}')
    print(f'Eprover: {provers[2]}')


def provers_to_csv(provers):
    print(','.join(provers))

In [8]:
def create_problem_folder(folder_name, expr, spec, tptp):
    os.makedirs(folder_name, exist_ok=True)
    with open(f'{folder_name}/expression.txt', 'w') as f:
        f.write(expr)
    with open(f'{folder_name}/specification.txt', 'w') as f:
        f.write(spec)
    with open(f'{folder_name}/tptp.p', 'w') as f:
        f.write(tptp)
    vampire, eprover = use_provers(f'{folder_name}/tptp.p')
    with open(f'{folder_name}/vampire.txt', 'w') as f:
        f.write(vampire)
    with open(f'{folder_name}/eprover.txt', 'w') as f:
        f.write(eprover)

# Julia's problems

In [10]:
log = pm4py.read_xes('Data/Hospital Billing - Event Log.xes')

process_tree = pm4py.discover_process_tree_inductive(
    log, 0.0, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
expr, spec, tptp = TreeToTptp('A').tree_to_tptp(process_tree)
create_problem_folder("test", expr, spec, tptp)

parsing log, completed traces :: 100%|██████████| 100000/100000 [00:14<00:00, 6871.01it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=T

Seq2(1]Loop(2]l_s_A0, new, tau_A0[2), Xor2(2]x2_s_A0, tau_A1, And2(3]a2_s_A0, Xor2(4]x2_s_A1, tau_A2, Loop(5]l_s_A1, billed, tau_A3[5), x2_e_A1[4), Xor2(4]x2_s_A2, tau_A4, Seq2(5]And2(6]a2_s_A1, Xor2(7]x2_s_A3, tau_A5, Loop(8]l_s_A2, delete, tau_A6[8), x2_e_A3[7), Xor2(7]x2_s_A4, tau_A7, Seq2(8]Loop(9]l_s_A3, Xor3(10]x3_s_A5, Seq2(11]Xor2(12]x2_s_A6, tau_A8, Xor2(13]x2_s_A7, set_status, join_pat, x2_e_A7[13), x2_e_A7[12), Xor2(12]x2_s_A8, tau_A9, And2(13]a2_s_A2, Xor2(14]x2_s_A9, tau_A10, Loop(15]l_s_A4, code_error, tau_A11[15), x2_e_A9[14), Seq3(14]Xor2(15]x2_s_A10, tau_A12, Loop(16]l_s_A5, release, tau_A13[16), x2_e_A10[15), Xor2(15]x2_s_A11, tau_A14, Xor2(16]x2_s_A12, code_ok, manual, x2_e_A12[16), x2_e_A12[15), Xor2(15]x2_s_A13, tau_A15, And2(16]a2_s_A3, Xor2(17]x2_s_A14, tau_A16, Loop(18]l_s_A6, code_nok, tau_A17[18), x2_e_A14[17), Xor2(17]x2_s_A15, tau_A18, Seq2(18]Xor2(19]x2_s_A16, tau_A19, And2(20]a2_s_A4, Xor2(21]x2_s_A17, tau_A20, Loop(22]l_s_A7, reject, tau_A21[22), x2_e_A17